# Nathan Rout

### Import Libraries and define matricies

In [1]:
import pandas as pd
import numpy as np
from numpy import linalg as la
np.seterr(divide='ignore')

import cufflinks as cf
cf.set_config_file(offline=True)

A = np.array([[0,3,-1,8],[-1,11,-1,3],[2,-1,10,-1],[10,-1,2,0]])
b = np.array([[15],[25],[-11],[6]])

## LU Decomposition

function definition:

In [2]:
def isdiagdom(matrix, detail=False) :
    '''
    Determines if a given matrix is diagonally dominant.
    
    isdiagdom(matrix, detail=False)
    matrix = matrix to be assessed
    detail (optional) = returns further information if set to "True"
    '''
    
    # check matrix is square
    if len(matrix) == len(matrix[0]) :
        
        n = len(matrix)
        
        if detail : print('is a', n, 'X', n, 'matrix')
        
        for i  in range(0, n) : 
            tot = 0   
            diag = abs(matrix[i][i])
            
            # sum values in row
            for j  in range(0, n) :
                if i == j : tot += 0
                else : tot += abs(matrix[i][j])
                        
            if detail : 
                print ('Absolute value of diagnoal element on row', i, 'is', diag)
                print ('Sum of absolute non-diagonal elements on row', i, 'is', tot)
                
            # check for dominance       
            if diag <= tot :
                if detail : print ('Matrix is not strictly diagonally dominant')        
                return False
            
        # dominance check hasnt failed, therefore diagonally dominant
        if detail : print ('Matrix is strictly diagonally dominant')  
        return True
        
    else :
        print('matrix is not square!')
        

In [3]:
isdiagdom(A, True)

is a 4 X 4 matrix
Absolute value of diagnoal element on row 0 is 0
Sum of absolute non-diagonal elements on row 0 is 12
Matrix is not strictly diagonally dominant


False

function definitions:

In [4]:
import operator

def mxpivot(matrix) :
    '''
    Generates a Pivot for a given matrix. 
    finds the max value in each row and moves it to the diagonal 
    '''
    n = len(matrix)
    I = np.eye(n)
    
    for j in range(0, n):
        #find index of max
        imax = np.argmax(matrix, axis =1)[j]
        if j != imax:
            # Swap the value                                                                                                                                                                                                                            
            I[j][j], I[imax][j] = I[imax][j], I[j][j]
   
    return I
    
    

In [5]:
def LUDecomp(matrix, method = "D") :
    '''
    Decomposes a given matrix into L and U. 
    
    LUDecomp(matrix, method = "D")
    matrix = matrix to be decomposed
    method (optional) = method of decomposition "D" for Doolittle, "C" for Crout
    '''
    
    # check matrix is square
    if len(matrix) == len(matrix[0]) :
        
        n = len(matrix)
        L = np.zeros((n, n))
        U = np.zeros((n, n))
        
        # check if pivot required
        if (not isdiagdom(matrix)) : P = mxpivot(matrix)
        else : P = np.eye(n)

        M = P.dot(matrix)
 
        if method == "D" :

            for i in range (0, n) : 

                L[i][i] = 1

                #upper
                for j in range  (i, n) :
                    sumu = 0

                    for k in range (0, i) :
                        sumu += ( L[i][k] * U[k][j] )

                    U[i][j] = M[i][j] - sumu


                #lower
                for j in range (i, n) :
                    suml = 0

                    for k in range (0, i) :
                        suml += ( L[j][k] * U[k][i] )

                    L[j][i] = ( M[j][i] - suml ) / U[i][i]  
           
        elif method == "C" :

            for i in range (0, n) : 
                
                U[i][i] = 1

                #lower
            for j in range  (0, n) :
                
                for i in range (j, n) :
                    suml = 0

                    for k in range (0, j) :
                        suml += ( L[i][k] * U[k][j] )

                    L[i][j] = M[i][j] - suml


                #upper
                for i in range (j, n) :
                    sumu = 0

                    for k in range (0, j) :
                        sumu += ( L[j][k] * U[k][i] )

                    U[j][i] = ( M[j][i] - sumu ) / L[j][j]            
        
        else :
            print("Unknown Method")
            return None
        
        return P, L, U
    
    else :
        print('matrix is not square!')


In [6]:
def LUDSolve(matrix, result, method = "D") :
    '''
    Solves a linear system using LU decomposition
    
    LUDSolve(matrix, result, method = "D")
    
    for Ax=b
    
    matrix = A
    result = b
    method (optional) = method of decomposition "D" for Doolittle, "C" for Crout
    
    '''
   
    # call function to return pivot, lower, and upper matricies  
    P, L, U = LUDecomp(matrix, method)
    
    n = len(matrix)
    p = P.dot(result)

    #Lz = p : find z
    z = np.zeros((1,n)).T
    z[0] = p[0] / L[0][0]
    
    for i in range (1, n) :
        sum1 = 0
        
        for j in range (0, n) :
            sum1 += L[i][j] * z[j]
            
        z[i] = (p[i] - sum1) / L[i][i]
  
    #Uy = z : find y
    y = np.zeros((1,n)).T
    y[n-1] = z[n-1] / U[n-1][n-1]    
    
    for i in range (n-2, -1, -1) :
        sum2 = 0
        
        for j in range (0, n) :
            sum2 += U[i][j] * y[j]
            
        y[i] = (z[i] - sum2) / U[i][i]
    
    return (np.multiply(P,y).sum(1)).T

Determine L and U for matrix A:

In [7]:
DP, DL, DU = LUDecomp(A, "D")
print("L using Doolittle Method:\n", pd.DataFrame(DL))
print("\n")
print("U using Doolittle Method:\n", pd.DataFrame(DU))
print("\n")

CP, CL, CU = LUDecomp(A, "C")
print("L using Crout Method:\n", pd.DataFrame(CL))
print("\n")
print("U using Crout Method:\n", pd.DataFrame(CU))

L using Doolittle Method:
      0         1         2    3
0  1.0  0.000000  0.000000  0.0
1 -0.1  1.000000  0.000000  0.0
2  0.2 -0.073394  1.000000  0.0
3  0.0  0.275229 -0.081731  1.0


U using Doolittle Method:
       0     1         2         3
0  10.0  -1.0  2.000000  0.000000
1   0.0  10.9 -0.800000  3.000000
2   0.0   0.0  9.541284 -0.779817
3   0.0   0.0  0.000000  7.110577


L using Crout Method:
       0     1         2         3
0  10.0   0.0  0.000000  0.000000
1  -1.0  10.9  0.000000  0.000000
2   2.0  -0.8  9.541284  0.000000
3   0.0   3.0 -0.779817  7.110577


U using Crout Method:
      0    1         2         3
0  1.0 -0.1  0.200000  0.000000
1  0.0  1.0 -0.073394  0.275229
2  0.0  0.0  1.000000 -0.081731
3  0.0  0.0  0.000000  1.000000


Solve using LU decomposition:

In [8]:
print("x using Doolittle Method:\n", pd.DataFrame(LUDSolve(A, b, "D")))
print("\n")
print("x using Crout Method:\n", pd.DataFrame(LUDSolve(A, b, "C")))

x using Doolittle Method:
      0
0  1.0
1  2.0
2 -1.0
3  1.0


x using Crout Method:
      0
0  1.0
1  2.0
2 -1.0
3  1.0


## Gauss-Seidel Method

In [9]:
G1 = np.array([[0],[0],[0],[0]])
G2 = np.array([[1],[1],[1],[1]])

function definition:

In [10]:
def gaussseidel(matrix, result, guess, prtstp = False, acc = 1.0) :
    '''
    Solves a linear system using Gauss-Seidel method
    
    gaussseidel(matrix, result, guess, prtstp = False, acc = 1.0)
    
    for Ax=b
    
    matrix = A
    result = b
    guess = initial guess of x
    prtstp (optional) = If "True" each iteration step will be printed, if "False" only final result will be shown
    acc (optional) = acceleration factor for SOR
    '''
    
    n = len(matrix)

    
    # check if pivot required
    if (not isdiagdom(matrix)) and (matrix[0][0] == 0) : P = mxpivot(matrix)
    else : P = np.eye(n)
    
    M = P.dot(matrix)
    b = P.dot(result)
    linf = []
    conv = False
    v = (guess.astype(float)).copy()
    perc = 4 # percision
    e = 10 ** - perc  # tolerance
    counter = 0

    if prtstp == True : print(counter, ": ", v.T)
    
    # continue to iterate until coverge, but up to 100 iterations (to prevent infinite loops)
    while (not conv) and (counter < 100) :
        
        # store previous iteration. used to determine convergence
        vprev = v.copy()

        # calcualte next iteration
        for i in range (0, n) :
            sum = 0  
           
            for j in range (0, n) :
                if j != i : sum += M[i][j] * v[j]
                                    
            v[i] = v[i] * (1 - acc) + (acc / M[i][i]) * ( b[i] - sum )
        
        counter += 1   
        
        
        # calc infinity norm
        l = (abs(v - vprev)).max() / (abs(vprev)).max()
        linf.append(l)
        
        
        if prtstp == True : print(counter, ": ", np.around((np.multiply(P,v).sum(1)).T, perc+1), 
                                  " l_inf = ", np.around(l, perc+1))     
        
        # Check convergence.   
        if l < e : conv = True
                
             
                
    return np.around((np.multiply(P,v).sum(1)).T, perc), counter, linf

using x(0) = (0,0,0,0)

In [11]:
soln, iter, l_infG1 = gaussseidel(A, b, G1, True)
print("x using Gauss-Seidel method:\n", pd.DataFrame(soln))
print("\nafter {} iterations".format(iter))

0 :  [[0. 0. 0. 0.]]
1 :  [ 0.6      2.32727 -0.98727  0.87886]  l_inf =  inf
2 :  [ 1.03018  2.03694 -1.01446  0.98434]  l_inf =  0.18484
3 :  [ 1.00659  2.00356 -1.00253  0.99835]  l_inf =  0.01639
4 :  [ 1.00086  2.0003  -1.00031  0.99985]  l_inf =  0.00286
5 :  [ 1.00009  2.00002 -1.00003  0.99999]  l_inf =  0.00038
6 :  [ 1.00001  2.      -1.       1.     ]  l_inf =  4e-05
x using Gauss-Seidel method:
      0
0  1.0
1  2.0
2 -1.0
3  1.0

after 6 iterations


using x(0) = (1,1,1,1)

In [12]:
soln, iter, l_infG2 = gaussseidel(A, b, G2, True)
print("x using Gauss-Seidel method:\n", pd.DataFrame(soln))
print("\nafter {} iterations".format(iter))

0 :  [[1. 1. 1. 1.]]
1 :  [ 0.5      2.13636 -0.88636  0.96307]  l_inf =  1.88636
2 :  [ 0.99091  2.01958 -0.99992  0.99267]  l_inf =  0.22979
3 :  [ 1.00194  2.00218 -1.0009   0.99907]  l_inf =  0.00861
4 :  [ 1.0004   2.00021 -1.00015  0.9999 ]  l_inf =  0.00099
5 :  [ 1.00005  2.00002 -1.00002  0.99999]  l_inf =  0.00017
6 :  [ 1.00001  2.      -1.       1.     ]  l_inf =  2e-05
x using Gauss-Seidel method:
      0
0  1.0
1  2.0
2 -1.0
3  1.0

after 6 iterations


Large matrix example (Numerical analysis, Burden and Faires, chapter 7.4)

In [13]:
lA = np.array([
    [4,-1,0,0,-1,0,0,0,0,0,0,0],
    [-1,4,-1,0,0,0,0,0,0,0,0,0],
    [0,-1,4,-1,0,0,0,0,0,0,0,0],
    [0,0,-1,4,0,-1,0,0,0,0,0,0],
    [-1,0,0,0,4,0,-1,0,0,0,0,0],
    [0,0,0,-1,0,4,0,-1,0,0,0,0],
    [0,0,0,0,-1,0,4,0,-1,0,0,0],
    [0,0,0,0,0,-1,0,4,0,0,0,-1],
    [0,0,0,0,0,0,-1,0,4,-1,0,0],
    [0,0,0,0,0,0,0,0,-1,4,-1,0],
    [0,0,0,0,0,0,0,0,0,-1,4,-1],
    [0,0,0,0,0,-1,0,0,0,0,-1,4]])
lb = np.array([[220],[110],[110],[220],[110],[110],[110],[110],[220],[110],[110],[220]])

lG1 = np.array([[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]])
lG2 = np.array([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1]])
lG3 = np.array([[70],[70],[70],[70],[70],[70],[70],[70],[70],[70],[70],[70]])

soln, iter, l_inflG1 = gaussseidel(lA, lb, lG1, True)
print("x using Gauss-Seidel method:\n", pd.DataFrame(soln))
print("\nafter {} iterations".format(iter))

0 :  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
1 :  [55.      41.25    37.8125  64.45312 41.25    43.61328 37.8125  38.40332
 64.45312 43.61328 38.40332 75.50415]  l_inf =  inf
2 :  [75.625   55.85938 57.57812 80.29785 55.85938 57.17529 57.57812 60.66986
 80.29785 57.17529 60.66986 84.46129]  l_inf =  0.2949
3 :  [82.92969 62.62695 63.2312  85.10162 62.62695 63.94287 63.2312  64.60104
 85.10162 63.94287 64.60104 87.13598]  l_inf =  0.08649
4 :  [86.31348 64.88617 64.99695 87.23495 64.88617 65.459   64.99695 65.64874
 87.23495 65.459   65.64874 87.77694]  l_inf =  0.03883
5 :  [87.44308 65.61001 65.71124 87.79256 65.61001 65.86033 65.71124 65.90932
 87.79256 65.86033 65.90932 87.94241]  l_inf =  0.01287
6 :  [87.805   65.87906 65.91791 87.94456 65.87906 65.96347 65.91791 65.97647
 87.94456 65.96347 65.97647 87.98498]  l_inf =  0.00412
7 :  [87.93953 65.96436 65.97723 87.98517 65.96436 65.99041 65.97723 65.99385
 87.98517 65.99041 65.99385 87.99606]  l_inf =  0.00153
8 :  [87.98218 65.9898

In [14]:
soln, iter, l_inflG2 = gaussseidel(lA, lb, lG2, True)
print("x using Gauss-Seidel method:\n", pd.DataFrame(soln))
print("\nafter {} iterations".format(iter))

0 :  [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
1 :  [55.5     41.625   38.15625 64.78906 41.625   43.94727 38.15625 38.73682
 64.78906 43.94727 38.73682 75.67102]  l_inf =  74.67102
2 :  [75.8125  55.99219 57.69531 80.41064 55.99219 57.28687 57.69531 60.73947
 80.41064 57.28687 60.73947 84.50658]  l_inf =  0.29077
3 :  [82.99609 62.67285 63.27087 85.13943 62.67285 63.96973 63.27087 64.61908
 85.13943 63.96973 64.61908 87.1472 ]  l_inf =  0.08501
4 :  [86.33643 64.90182 65.01031 87.24501 64.90182 65.46602 65.01031 65.65331
 87.24501 65.46602 65.65331 87.77983]  l_inf =  0.03833
5 :  [87.45091 65.61531 65.71508 87.79528 65.61531 65.86215 65.71508 65.91049
 87.79528 65.86215 65.91049 87.94316]  l_inf =  0.0127
6 :  [87.80765 65.88068 65.91899 87.94528 65.88068 65.96394 65.91899 65.97678
 87.94528 65.96394 65.97678 87.98518]  l_inf =  0.00406
7 :  [87.94034 65.96483 65.97753 87.98537 65.96483 65.99054 65.97753 65.99393
 87.98537 65.99054 65.99393 87.99612]  l_inf =  0.00151
8 :  [87.98242 65

In [15]:
soln, iter, l_inflG3 = gaussseidel(lA, lb, lG3, True)
print("x using Gauss-Seidel method:\n", pd.DataFrame(soln))
print("\nafter {} iterations".format(iter))

0 :  [[70. 70. 70. 70. 70. 70. 70. 70. 70. 70. 70. 70.]]
1 :  [90.      67.5     61.875   87.96875 67.5     66.99219 61.875   61.74805
 87.96875 66.99219 61.74805 87.18506]  l_inf =  0.28571
2 :  [88.75    65.15625 65.78125 88.19336 65.15625 64.98535 65.78125 65.5426
 88.19336 64.98535 65.5426  87.63199]  l_inf =  0.0434
3 :  [87.57812 65.83984 66.0083  87.74841 65.83984 65.82275 66.0083  65.86369
 87.74841 65.82275 65.86369 87.92161]  l_inf =  0.0132
4 :  [87.91992 65.98206 65.93262 87.93884 65.98206 65.95063 65.93262 65.96806
 87.93884 65.95063 65.96806 87.97967]  l_inf =  0.00389
5 :  [87.99103 65.98091 65.97994 87.98264 65.98091 65.98768 65.97994 65.99184
 87.98264 65.98768 65.99184 87.99488]  l_inf =  0.00081
6 :  [87.99046 65.9926  65.99381 87.99537 65.9926  65.9968  65.99381 65.99792
 87.99537 65.9968  65.99792 87.99868]  l_inf =  0.00016
7 :  [87.9963  65.99753 65.99822 87.99876 65.99753 65.99917 65.99822 65.99946
 87.99876 65.99917 65.99946 87.99966]  l_inf =  7e-05
x using Ga

## Successive Over Relaxation

Using acceleration factor 1.1

In [16]:
soln, iter, l_inf = gaussseidel(A, b, G1, True, 1.1)
print("x using Gauss-Seidel method:\n", pd.DataFrame(soln))
print("\nafter {} iterations".format(iter))

0 :  [[0. 0. 0. 0.]]
1 :  [ 0.66     2.566   -1.07294  0.8565 ]  l_inf =  inf
2 :  [ 1.11231  1.99039 -1.03426  1.01361]  l_inf =  0.22432
3 :  [ 0.99525  1.99298 -0.9948   1.00225]  l_inf =  0.05881
4 :  [ 0.99856  2.0004  -0.99991  0.99962]  l_inf =  0.00372
5 :  [ 1.00017  2.0001  -1.00008  0.99999]  l_inf =  0.0008
6 :  [ 1.00001  1.99999 -1.       1.00001]  l_inf =  8e-05
x using Gauss-Seidel method:
      0
0  1.0
1  2.0
2 -1.0
3  1.0

after 6 iterations


Large matrix example

In [17]:
soln, iter, l_inf = gaussseidel(lA, lb, lG1, True, 1.1)
print("x using Gauss-Seidel method:\n", pd.DataFrame(soln))
print("\nafter {} iterations".format(iter))

0 :  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
1 :  [60.5     46.8875  43.14406 72.36462 46.8875  50.15027 43.14406 44.04132
 72.36462 50.15027 44.04132 86.40269]  l_inf =  inf
2 :  [80.23812 59.49135 62.19599 84.15876 59.49135 60.49    62.19599 66.24136
 84.15876 60.49    66.24136 86.71085]  l_inf =  0.25694
3 :  [85.19643 64.83378 65.00335 86.59479 64.83378 66.23094 65.00335 65.68486
 86.59479 66.23094 65.68486 88.10576]  l_inf =  0.06621
4 :  [87.63894 65.74325 65.64263 88.10575 65.74325 65.91932 65.64263 66.03841
 88.10575 65.91932 66.03841 87.9778 ]  l_inf =  0.02772
5 :  [87.89489 65.89849 66.0369  87.97739 65.89849 66.01241 66.0369  65.99347
 87.97739 66.01241 65.99347 88.00384]  l_inf =  0.00448
6 :  [87.95468 66.00784 65.99225 88.00354 66.00784 65.99794 65.99225 66.00114
 88.00354 65.99794 66.00114 87.99936]  l_inf =  0.00124
7 :  [88.00884 65.99952 66.00162 87.99952 65.99952 66.00039 66.00162 65.99982
 87.99952 66.00039 65.99982 88.00012]  l_inf =  0.00062
8 :  [87.99885 66.000

## Analysis

### Initial Guess

When solving simple systems, such as the 4x4 case, the initial guess did not appear to have much of an impact on the rate of convergence. The initial guess of "0's" meant that the infinity norm (l_inf) could not be calculated on the first iteration, however the second iteration had a much lower l_inf than the initial guess of "1's". Beyond this, however, the "1's" showed a marginally better convergence.
In this case the same number of iterations were required for both guesses. However, it could suggest that in a much larger and more complex system that having an initial guess close to the solution (i.e. an "educated guess") could yield faster convergence and therefore fewer iterations

By using a slightly more complex system, 12x12, the impact of supplying an "educated guess" was more significant. The use of "0's" or "1's" had a minimal impact on the rate of convergence and both took 10 iterations to complete. By supplying a guess, that wasnt far from the expected result (i.e. "70's"), was able to meet the required percision after 6 iterations (see plots below).
This would suggest, that some understanding of the system being solved and a rough idea of the order of the solution will have a significant impact on the efficiency.

In [18]:
df1 = pd.DataFrame(l_infG1, columns = ['Zeroes'])
df2 = pd.DataFrame(l_infG2, columns = ['Ones'])
df = df1.append(df2)

df.iplot(kind='line', title='Convergence of a 4x4 system, with initial guess', xTitle='Iteration', yTitle='l_inf')
df.iplot(kind='line', xTitle='Iteration', yTitle='l_inf', subplots=True)

In [19]:
df1 = pd.DataFrame(l_inflG1, columns = ['Zeroes'])
df2 = pd.DataFrame(l_inflG2, columns = ['Ones'])
df3 = pd.DataFrame(l_inflG3, columns = ['EduGuess'])
df = df1.append(df2)
df = df.append(df3)

df.iplot(kind='line', title='Convergence of a 12x12 system, with initial guess', xTitle='Iteration', yTitle='l_inf')
df.iplot(kind='line', xTitle='Iteration', yTitle='l_inf', subplots=True)

### Acceleration Factor

With a range of acceleration factors, for both the 4x4 and 12x12 systems, we can clearly see the impact below and determine the optimal factor.
In the 4x4 case, acceleration factors of 1, 1.05, and 1.1 resulted in the least number of iterations required (6). The rate of convergence decreased as the factor became larger that 1.1. What is interesting is that for the simple case, the acceleration factor had no material impact over the standard case of 1 - infact l_inf was consistently smaller with a factor of 1.
For a more complex system, we can see the acceleration factor starting to play a bigger role. We can see that the acceleration factor of 1.05 and 1.1 resulted in the least number of iterations (9), where the standard case required more iterations (10). We can also observe that l_inf was consistantly lower for 1.1 than 1.05.

In [20]:
ac = np.linspace(0.9,1.3,9)
data = {}

for i in range (0, len(ac)) :
    soln, iter, l_infacc = gaussseidel(A, b, G1, acc = ac[i])
    data[ac[i]] = l_infacc
    
df = pd.DataFrame.from_dict(data, orient='index')
df = df.transpose()
 
df.iplot(kind='line', title='Convergence of a 4x4 system, with acceleration factor', xTitle='Iteration', yTitle='l_inf')    
df.iplot(kind='line', xTitle='Iteration', yTitle='l_inf', subplots=True)

In [21]:
ac = np.linspace(0.9,1.3,9)
data = {}

for i in range (0, len(ac)) :
    soln, iter, l_infacc = gaussseidel(lA, lb, lG1, acc = ac[i])
    data[ac[i]] = l_infacc
    
df = pd.DataFrame.from_dict(data, orient='index')
df = df.transpose()
 
df.iplot(kind='line', title='Convergence of a 4x4 system, with acceleration factor', xTitle='Iteration', yTitle='l_inf')    
df.iplot(kind='line', xTitle='Iteration', yTitle='l_inf', subplots=True)

### Conclusion

Based on both the initial guess and the use of an acceleration factor, we have seen that, for simple systems, that is no material impact of efficiency - the solution was obtained with the same number of iterations. However, with a more complex system, the choice of initial guess and the use of an acceleration factor was, independantly, able to reduce the number of iterations required to reach a solution within the desired tolerance. More examples than those above would be required to arrive at a definite conclusion, but the results would suggest that such techniques (acceleration factor and initial guess) improve the scalability of the Gauss-Seidel method.